In [ ]:
from Bio import Entrez
import pandas as pd
from tqdm import tqdm
import http

path = "E:/YandexDisk/Work/bbd/epi_pred"

Entrez.email = "alena.kalyakulina@itmm.unn.ru"
Entrez.api_key = 'd4cd80314ce06cf5cb310beba9e013400808'

platforms = [
    "GPL13534", #450k
    #"GPL16304", #450k
    "GPL21145", #EPIC
    #"GPL23976", #EPIC
    #"GPL29753", #EPIC
    #"GPL33022", #EPICv2 
    #"GPL34394", #MSA
    ]

for platform in platforms:
    all_results = []
    search_term = f"{platform}[Platform] AND Homo sapiens[Organism]" # Выберет GPL, GSE и все GSM
    #search_term = f"{platform}[Platform] AND Homo sapiens[Organism] AND GSE[Filter]" # Выберет GSE
    handle = Entrez.esearch(db="gds", term=search_term, retmax=1000000)
    record = Entrez.read(handle)
    entry_ids = record["IdList"]

    for entry_id in tqdm(entry_ids):
        while True:
            try:
                handle = Entrez.esummary(db="gds", id=entry_id)
            except http.client.RemoteDisconnected:
                continue
            break
        while True:
            try: 
                summary = Entrez.read(handle)[0]
            except http.client.IncompleteRead:
                continue
            break
        if summary['entryType'] == 'GSM':
            all_results.append({
                "Accession": summary['Accession'],
                "Source": summary['summary'],
                "GPL": "GPL" + (";" + "GPL").join(summary['GPL'].split(";")),
                "GSE": "GSE" + (";" + "GSE").join(summary['GSE'].split(";")),
                "Organism": summary['taxon'],
                "Date": summary['PDAT']
            })
    df = pd.DataFrame(all_results)
    df.to_excel(f"{path}/{platform}_geo_datasets.xlsx", index=False)